# Using the brokers opinion data
In this document I process the data collected and model it both by itself and with the data of the stock prices.

<br>

Overview, the use of the brokers opinion data does indeed result in improved performance than using just the stock price data. I also appears that the use of the brokers data alone is better than using the stock price data with or without the brokers data. And individual models perform better than collective models.
This is promising that the use of news articles in the third part of the project will give even better results.


In [13]:
import pandas as pd
import re
import numpy as np
import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import MinMaxScaler
import datetime as dt

from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Lambda, Dropout, BatchNormalization

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))


from sklearn.metrics import classification_report, confusion_matrix



## Processing 
This section of the code produces the data that can easily be modelled on. 

<br>

This fist part of the code creates a list of the names of the FTSE 100 companies that are used to open the stock price data for each of the companies. And opens the brokers opinion data set.


In [15]:

    
File = open('stocks_name.txt','r')

Name = []
Codes = []
Type = []
Add = []

for line in File:
    line = re.sub('\n', '', line)
    line = line.split('\t')
    Name.append(re.sub(' ', ' ',line[0].lower())+' ')
    Type.append(line[2])
    Add.append(line[1].lower()+ ' ')
    if line[1][2:4] == '.A':
        Codes.append('BT-A.L')
    elif line[1][2] == '.':
        Codes.append(line[1] + 'L')
    else:
        Codes.append(line[1] + '.L')


file = open('Brokers3.csv','r')

number_per = []
lines = []
for line in file:
    line = line.split('], ')
    number_per.append(len(line))
    lines.append(line)

This part of the code splits the data from the brokers data into 5 categories; the date the opinion was released, the company that released the opinion, the opinion ('Buy', 'Sell', ect.), the previously predicted price that the stock will go to, the current predicted future price, and the class that is if this is a reiteration, downgrade or upgrade based on previous opinions. 

<br>
In this project I am only concerned with the opinion and the date that the opinion was made.

In [43]:

dates = []
companies= []
opinions = []
pre_preds = []
preds = []
Classes = []


for i in range(len(lines)):
    
    date = []
    company = []
    opinion = []
    pre_pred = []
    pred = []
    Class = []
    
    for j in range(len(lines[i])):
        
        date.append(re.sub('\[', '', lines[i][j].split(',')[0]))
        company.append(lines[i][j].split(',')[1])
        opinion.append(lines[i][j].split(',')[2])
        pre_pred.append(lines[i][j].split(',')[3])
        pred.append(lines[i][j].split(',')[4])
        Class.append(lines[i][j].split(',')[5])
        
    dates.append(date)
    companies.append(company)
    opinions.append(opinion)
    pre_preds.append(pre_pred)
    preds.append(pred)
    Classes.append(Class)
print(opinions[0])

[" 'Neutral'", " 'Buy'", " 'Buy'", " 'Buy'", " 'Buy'", " 'Buy'", " 'Overweight'", " 'Buy'", " 'Buy'", " 'Equal-weight'", " 'Neutral'", " 'Overweight'", " 'Buy'", " 'Buy'", " 'Buy'", " 'Overweight'", " 'Buy'", " 'Buy'", " 'Buy'", " 'Buy'", " 'Buy'", " 'Buy'", " 'Buy'", " 'Overweight'", " 'Buy'", " 'Buy'", " 'Buy'", " 'Buy'", " 'Buy'", " 'Overweight'", " 'Overweight'", " 'Buy'", " 'Outperform'", " 'Equal-weight'", " 'Overweight'", " 'Neutral'", " 'Overweight'", " 'Outperform'", " 'Outperform'", " 'Buy'", " 'Overweight'", " 'Buy'", " 'Overweight'", " 'Buy'", " 'Overweight'", " 'Buy'", " 'Overweight'", " 'Buy'", " 'Buy'", " 'Overweight'", " 'Buy'", " 'Buy'", " 'Overweight'", " 'Hold'", " 'Buy'", " 'Neutral'", " 'Neutral'", " 'Buy'", " 'Buy'", " 'Hold'", " 'Equal-weight'", " 'Buy'", " 'Hold'", " 'Overweight'"]


This creates a list of the opinions, 
I group data into 5 categories based on the implied meaning of the name,<br>
-- sell-(sell, strong sell)<br>
-- underperform-(underperform, moderate sell, weak sell, underwieght, reduce)<br>
-- hold -(hold, neutral,equal weight)<br>
-- outperform-(moderate buy, accumulate, add, over-weight, add,House Stock, Sector Perform,Outperform)<br>
-- buy-(buy, strong buy,Top Pick)<br> 
This is done so an appropriate number of instances of these events can be seen in my training and testing data set.

I also keep the data in a 100 element list one for each of the FTSE 100.

In [42]:


Views = []
for i in range(100):
    
    Opinions = []
    
    for op in opinions[i]:
        op = op[2:-1]
        
        if op == 'Buy':
            Opinions.append('Buy')
        elif op == 'Top Pick':
            Opinions.append('Buy')
        elif op == 'Strong Buy':
            Opinions.append('Buy')
            
        elif op == 'Overweight':
            Opinions.append('Outperform')
        elif op == 'Add':
            Opinions.append('Outperform')
        elif op == 'House Stock':
            Opinions.append('Outperform')
        elif op == 'Sector Perform':
            Opinions.append('Outperform')
        elif op == 'Outperform':
            Opinions.append('Outperform')
        elif op == 'Accumulate':
            Opinions.append('Outperform')
        elif op == 'Speculative Buy':
            Opinions.append('Outperform')
        elif op == 'Market Perform':
            Opinions.append('Outperform')
        elif op == 'Corporate':
            Opinions.append('Outperform')
                        
        elif op == 'Hold':
            Opinions.append('Hold')
        elif op == 'Neutral':
            Opinions.append('Hold')
        elif op == 'Equal-weight':
            Opinions.append('Hold')
        elif op == 'Not Rated':
            Opinions.append('Hold')
        elif op == 'arnier & Co':
            Opinions.append('Hold')
        elif op == 'In-Line':
            Opinions.append('Hold')
            
        elif op == 'Underperform':
            Opinions.append('Underperform')
        elif op == 'Underweight':
            Opinions.append('Underperform')
        elif op == 'Reduce':
            Opinions.append('Underperform')
            
        elif op == 'Sell':
            Opinions.append('Sell')
        elif op == 'Suspended':
            Opinions.append('Sell')
        elif op == 'Strong Sell':
            Opinions.append('Sell')
            
        else:
            print('Error ', op, 'not recognised!')
        
    Views.append(Opinions)
print('Examples : ')
print(Views[0])

Examples : 
['Hold', 'Buy', 'Buy', 'Buy', 'Buy', 'Buy', 'Outperform', 'Buy', 'Buy', 'Hold', 'Hold', 'Outperform', 'Buy', 'Buy', 'Buy', 'Outperform', 'Buy', 'Buy', 'Buy', 'Buy', 'Buy', 'Buy', 'Buy', 'Outperform', 'Buy', 'Buy', 'Buy', 'Buy', 'Buy', 'Outperform', 'Outperform', 'Buy', 'Outperform', 'Hold', 'Outperform', 'Hold', 'Outperform', 'Outperform', 'Outperform', 'Buy', 'Outperform', 'Buy', 'Outperform', 'Buy', 'Outperform', 'Buy', 'Outperform', 'Buy', 'Buy', 'Outperform', 'Buy', 'Buy', 'Outperform', 'Hold', 'Buy', 'Hold', 'Hold', 'Buy', 'Buy', 'Hold', 'Hold', 'Buy', 'Hold', 'Outperform']


Now I wish to make a list that contains the date and the opinion in a one-hot-encoded way. As can be seen below

In [44]:
BROKER = []

for target in range(100):
    
    dat = dates[target]
    da = []
    for d in dat:
        d = d[1:-1]
        da.append(datetime.datetime.strptime(d, '%d-%b-%y'))


    Buy = []
    Outperform = []
    Hold = []
    Underperform = []
    Sell = []
    for i in range(len(dat)):
        if Views[target][i] == 'Buy':
            Buy.append(1)
            Outperform.append(0)
            Hold.append(0)
            Underperform.append(0)
            Sell.append(0)
        elif Views[target][i] == 'Outperform':
            Buy.append(0)
            Outperform.append(1)
            Hold.append(0)
            Underperform.append(0)
            Sell.append(0)
        elif Views[target][i] == 'Hold':
            Buy.append(0)
            Outperform.append(0)
            Hold.append(1)
            Underperform.append(0)
            Sell.append(0)
        elif Views[target][i] == 'Underperform':
            Buy.append(0)
            Outperform.append(0)
            Hold.append(0)
            Underperform.append(1)
            Sell.append(0)
        elif Views[target][i] == 'Sell':
            Buy.append(0)
            Outperform.append(0)
            Hold.append(0)
            Underperform.append(0)
            Sell.append(1)
            
    data = []
    for i in range(len(dat)):
        data.append([da[i], Buy[i], Outperform[i], Hold[i], Underperform[i], Sell[i]])
        
    BROKER.append(data)


DF = pd.DataFrame(BROKER[10], columns=['Date','Buy', 'Outperform', 'Hold', 'Underperform', 'Sell'])
DF = DF.sort_values('Date')
DF.set_index('Date', inplace=True)
DF = DF.truncate(before='2014-01-02')
DF = DF.groupby(DF.index).sum()
DF.head(10)

,Buy,Outperform,Hold,Underperform,Sell
Date,,,,,
2014-01-13,0,0,1,0,0
2014-01-15,0,0,0,1,0
2014-01-31,0,0,0,1,0
2014-02-03,1,0,0,0,0
2014-02-05,0,0,1,0,0
2014-02-11,1,0,0,0,0
2014-02-19,1,0,0,0,0
2014-02-20,1,0,0,0,0
2014-02-21,1,0,0,0,0



Now I wish to create a data frame that contains the opinions and matches them up with the date they where released. This creates a data point one per day so if two articles are released on the same day they become one point in my data frame, as seen above.
<br>

As I do not have any stock price data for before 2014 I remove all the brokers data from before then this is about 5% of the data.
<br>

I then go on to open the stock price data and join the two data frames together based on the date. And fill in all the missing data form the brokers data set as they aren't released as often as the stock data is.

<br>

As the brokers opinions are relevant for more than just the day they are released. This is because they are longer term predictions for the price, I propagate the data through the data frame. The amount of time this is done for is given by the variable day, and is currently set at 100 days. 

<br>
An example of the data processing can be seen below.

In [47]:
DATA = []

for target in range(len(Codes)):
    
    DF = pd.DataFrame(BROKER[target], columns=['Date','Buy', 'Outperform', 'Hold', 'Underperform', 'Sell'])
    DF = DF.sort_values('Date')
    DF.set_index('Date', inplace=True)
    DF = DF.truncate(before='2014-01-02')
    # Deals with two opinions being posted on the same day
    DF = DF.groupby(DF.index).sum()

    df = pd.read_csv('csv/' + Codes[target] + '.csv')
    # Sort DataFrame by date
    df = df.sort_values('Date')
    df.set_index('Date', inplace=True)
    df = df.dropna()

    # Joining the datasets 
    broker_data = df.join(DF, how='outer')
    broker_data.fillna(0)
    
    ###################
    days = 100
    ###################
    
    broker_data['Buy1'] = broker_data['Buy'][::1].rolling(days, min_periods=0).sum()[::-1]
    broker_data['Outperform1'] = broker_data['Outperform'][::1].rolling(days, min_periods=0).sum()[::-1]
    broker_data['Hold1'] =broker_data['Hold'][::1].rolling(days, min_periods=0).sum()[::-1]
    broker_data['Underperform1'] = broker_data['Underperform'][::1].rolling(days, min_periods=0).sum()[::-1]
    broker_data['Sell1'] = broker_data['Sell'][::1].rolling(days, min_periods=0).sum()[::-1]

    broker_data = broker_data.drop(columns=['Buy','Outperform','Hold','Underperform', 'Sell'])
    
    DATA.append(broker_data.values)


DF = pd.DataFrame(DATA[10], columns=['Open', 'High', 'Low', 'Close', 'Adj Close','Volume','Buy', 'Outperform', 'Hold', 'Underperform', 'Sell'])
DF.tail()

,Open,High,Low,Close,Adj Close,Volume,Buy,Outperform,Hold,Underperform,Sell
1573,496.100006,523.599976,480.000000,523.599976,523.599976,14690819.0,2.0,1.0,1.0,0.0,0.0
1574,505.600006,537.599976,498.399994,537.599976,537.599976,16495524.0,2.0,1.0,1.0,0.0,0.0
1575,523.599976,523.599976,499.000000,516.200012,516.200012,8943020.0,2.0,1.0,1.0,0.0,0.0
1576,514.599976,520.200012,498.100006,508.399994,508.399994,7855795.0,2.0,1.0,1.0,0.0,0.0
1577,513.799988,523.000000,498.598999,521.799988,521.799988,12679800.0,2.0,1.0,1.0,0.0,0.0


## Window function
This function creates overlapping windows of the data. That is it takes 'size' number of days (the size that the window should be) for each data point. It then takes the future value of the stock and compares the change from the last day of the data window. I will be using the closing price as the value I am trying to predict.

<br>
I have allowed for both using and not using scaling of the data by either using or removing the scaling from this function.
I also allow for the use of just the brokers data or just the stock price data or both 

In [48]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


def window(stock, brokers, size, delay):
    
    windows = []
    binary_pred = []
    pred = []
        
    
    for i in range(len(stock) - delay - size):
        
        scaler.fit(stock[i: i + size])
        
        win = []
        for j in range(size):
            # comment top line to only include brokers opinions
            #win.append(np.concatenate(scaler.transform(stock[i+j:i+j+1])))
            win.append(brokers[i+j])
            
        windows.append(np.concatenate(win))
        
        pred.append(scaler.transform(stock[i+size:i + size + delay])[-1][-3] - scaler.transform(stock[i:i + size])[-1][-3]) # The actual value scaled appropriatly
        
        #Binary pred catagory
        if scaler.transform(stock[i+size:i + size + delay])[-1][-3] < scaler.transform(stock[i:i + size])[-1][-3]:
            binary_pred.append(1)
        else:
            binary_pred.append(0)
    
        
    return windows, binary_pred, pred


## Now lets look at using the stock price and brokers for each company
This part of the code models the data. I use four models for this part; SVM, logistic regression, XGBoost and an LSTM.

<br>
To avoid my model form being able to see future data and to keep the time series features I do not shuffle the data and I also do not use conventional validation techniques such as k-fold validation. I also train on the first 1000 data point that represents about 4.5 years of my data and test on 1.5 years 


In [27]:
Svm = svm.SVC(C=1)
mod = XGBClassifier()

SVM = []
LOG = []
XGB = []
LSTm = []
SPLIT = []


for i in range(54):

    DF = pd.DataFrame(DATA[i], columns=['Open', 'High', 'Low', 'Close', 'Adj Close','Volume','Buy', 'Outperform', 'Hold', 'Underperform', 'Sell'])

    DF = DF.dropna()

    
    Brokers = DF[['Buy','Outperform', 'Hold', 'Underperform', 'Sell']].values

    Stock = DF[['Open', 'High', 'Low', 'Close', 'Adj Close','Volume']].values
    
    
    windows, binary_pred, pred = window(Stock, Brokers, 10, 10)
    
    
    X_train = windows[:1000]
    X_test = windows[1000:]
    y_train = binary_pred[:1000]
    y_test = binary_pred[1000:]
    
    if len(y_test) > 0:
    
        split = 0
        for i in y_test:
            if i == 0:
                split+=1
        SPLIT.append(split/len(y_test))

        print('Split : ', split/len(y_test))

        Svm.fit(X_train, y_train)
        SVm = Svm.score(X_test, y_test) 
        print('SVM : ', SVm)

        clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)
        log = clf.score(X_test, y_test)
        print('log reg : ', log)

        mod.fit(np.array(X_train), np.array(y_train))
        xgb = mod.score(np.array(X_test), np.array(y_test))
        print('XGBoost : ', xgb)

        # reshape input to be 3D [samples, timesteps, features]
        train_X = np.array(X_train).reshape((np.array(X_train).shape[0], 1,np.array(X_train).shape[1]))
        test_X = np.array(X_test).reshape((np.array(X_test).shape[0], 1, np.array(X_test).shape[1]))


        Model = Sequential()
        Model.add(LSTM(180, return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])))
        Model.add(LSTM(units=90, return_sequences=True))
        #model.add(BatchNormalization())
        Model.add(LSTM(units=30))
        Model.add(Dense(units=1, activation='relu'))

        Model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        Model.fit(train_X,np.array(y_train), epochs=10, batch_size=32,shuffle = False,validation_split=0.2, verbose=0)

        lstm = Model.evaluate(np.array(test_X),np.array(y_test), verbose=0)[1]

        print('LSTM : ', lstm)

        SVM.append(SVm)
        LOG.append(log)
        XGB.append(xgb)
        LSTm.append(lstm)

Split :  0.9774011299435028
SVM :  0.9774011299435028
log reg :  0.9698681732580038
XGBoost :  0.9378531073446328
LSTM :  0.9774011373519897
Split :  0.5350089766606823
SVM :  0.5314183123877917
log reg :  0.4865350089766607
XGBoost :  0.4631956912028725
LSTM :  0.53500896692276
Split :  0.5116696588868941
SVM :  0.5116696588868941
log reg :  0.5152603231597845
XGBoost :  0.5314183123877917
LSTM :  0.5116696357727051
Split :  0.4793536804308797
SVM :  0.4793536804308797
log reg :  0.48473967684021546
XGBoost :  0.4793536804308797
LSTM :  0.479353666305542
Split :  0.526032315978456
SVM :  0.5278276481149012
log reg :  0.5224416517055656
XGBoost :  0.4542190305206463
LSTM :  0.5170556306838989
Split :  0.5008976660682226
SVM :  0.4129263913824057
log reg :  0.5116696588868941
XGBoost :  0.4793536804308797
LSTM :  0.5008976459503174
Split :  0.5080789946140036
SVM :  0.4919210053859964
log reg :  0.5044883303411131
XGBoost :  0.5224416517055656
LSTM :  0.5727109313011169
Split :  0.46428

### Results

These are the results for using both the stock price and brokers opinions

As can be seen the LSTM does out perform the other models as hoped due to its increased complexity 

In [23]:
print(len(SVM))
results = pd.DataFrame({'SVM' : SVM, 'Logistic reg' : LOG, 'XBGoost' : XGB, 'LSTM': LSTm}, index = Codes[:54])
#results.to_csv('Brokers_stock_individual.csv')
results.head()

54


,SVM,Logistic reg,XBGoost,LSTM
III.L,0.977401,0.969868,0.996234,0.979284
ADM.L,0.531418,0.488330,0.511670,0.542190
AAL.L,0.511670,0.511670,0.526032,0.511670
ANTO.L,0.479354,0.479354,0.490126,0.479354
AHT.L,0.509874,0.513465,0.513465,0.526032


In [24]:
print('Mean')
print(results.mean())
print('Variance')
print(results.var())

Mean
SVM             0.512024
Logistic reg    0.513030
XBGoost         0.514192
LSTM            0.518968
dtype: float64
Variance
SVM             0.005983
Logistic reg    0.005285
XBGoost         0.005962
LSTM            0.006867
dtype: float64


These are the results when using only the brokers opinions

In [29]:
results = pd.DataFrame({'SVM' : SVM, 'Logistic reg' : LOG, 'XBGoost' : XGB, 'LSTM': LSTm}, index = Codes[:54])
results.to_csv('Brokers_individual.csv')
results.head()

,SVM,Logistic reg,XBGoost,LSTM
III.L,0.977401,0.969868,0.937853,0.977401
ADM.L,0.531418,0.486535,0.463196,0.535009
AAL.L,0.511670,0.515260,0.531418,0.511670
ANTO.L,0.479354,0.484740,0.479354,0.479354
AHT.L,0.527828,0.522442,0.454219,0.517056


In [30]:
print('Mean')
print(results.mean())
print('Variance')
print(results.var())

Mean
SVM             0.513861
Logistic reg    0.516692
XBGoost         0.508576
LSTM            0.528887
dtype: float64
Variance
SVM             0.006480
Logistic reg    0.006549
XBGoost         0.005689
LSTM            0.006568
dtype: float64


## Now combining all the companies

Instead of building one model for each of the companies I combine the data sets. To do this I only take the first 1000 data point of each of the sets leaving the rest for future prediction. This avoids my models being able to see future data of other companies that would not be correct to do. 

In [33]:
delay = 10
days = 10    

X_train = []
X_test = []
y_train = []
y_test = []
binary_train = []
binary_test = []
ALL = []


for i in range(100):

    DF = pd.DataFrame(DATA[i], columns=['Open', 'High', 'Low', 'Close', 'Adj Close','Volume','Buy', 'Outperform', 'Hold', 'Underperform', 'Sell'])

    DF = DF.dropna()
    
    Brokers = DF[['Buy','Outperform', 'Hold', 'Underperform', 'Sell']].values

    Stock = DF[['Open', 'High', 'Low', 'Close', 'Adj Close','Volume']].values
    
    windows, binary_pred, pred = window(Stock, Brokers, 10, 10)
    
    print(len(windows),len(pred), len(binary_pred))
    
    y_train = y_train + pred[:1000]
    y_test = y_test + pred[1000:]
    
    binary_train = binary_train + binary_pred[:1000]
    binary_test = binary_test + binary_pred[1000:]
        
    X_train = X_train + windows[:1000]
    X_test = X_test + windows[1000:]
    
    ALL = ALL + binary_pred[1000-delay:-delay]

1531 1531 1531
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1252 1252 1252
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1470 1470 1470
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
94 94 94
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1557 1557 1557
1

In [34]:
print(len(X_train),len(y_train), len(binary_train))
print(len(X_test),len(y_test), len(binary_test))

split = 0
for i in binary_test:
    if i == 0:
        split+=1
print('Test : ', split/len(binary_test))

split = 0
for i in binary_train:
    if i == 0:
        split+=1
print('trian : ', split/len(binary_train))

split = 0
for i in binary_pred:
    if i == 0:
        split+=1

print('All : ', split/len(binary_pred))

99094 99094 99094
54480 54480 54480
Test :  0.5149229074889868
trian :  0.5410922962036047
All :  0.50417469492614


In [35]:
#Svm.fit(X_train, binary_train)
#SVm = Svm.score(X_test, binary_test) 
#print('SVM : ', SVm)
    
clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, binary_train)
print('log reg : ', clf.score(X_test, binary_test))

mod = XGBClassifier()
mod.fit(np.array(X_train), np.array(binary_train))
print('XGBoost : ', mod.score(np.array(X_test), np.array(binary_test)))

y_pred = clf.predict(X_test)

y = []
for i in range(len(y_pred)):
    if y_pred[i] == 0:
        y.append(0)
    else:
        y.append(1)
        
print('Confusion Matrix')
print(confusion_matrix(binary_test, y))
print('Classification Report')

a=0
for i in range(len(y)):
    if y[i] == binary_test[i]:
        a+=1
print('Accuracy : ', a/len(y_pred))
               
target_names = ['Increase', 'decrease']
print(classification_report(binary_test, y, target_names=target_names))

log reg :  0.5149045521292217
XGBoost :  0.516409691629956
Confusion Matrix
[[28050     3]
 [26425     2]]
Classification Report
Accuracy :  0.5149045521292217
              precision    recall  f1-score   support

    Increase       0.51      1.00      0.68     28053
    decrease       0.40      0.00      0.00     26427

    accuracy                           0.51     54480
   macro avg       0.46      0.50      0.34     54480
weighted avg       0.46      0.51      0.35     54480



In [38]:
print(np.array(X_train).shape)
# reshape input to be 3D [samples, timesteps, features]
train_X = np.array(X_train).reshape((np.array(X_train).shape[0], 1,np.array(X_train).shape[1]))
test_X = np.array(X_test).reshape((np.array(X_test).shape[0], 1, np.array(X_test).shape[1]))

Model = Sequential()
Model.add(LSTM(180, return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2])))
Model.add(LSTM(units=90, return_sequences=True))
#Model.add(BatchNormalization())
Model.add(LSTM(units=30))
Model.add(Dense(units=1, activation='relu'))

Model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
Model.summary()

Model.fit(train_X,np.array(binary_train), epochs=10, batch_size=32, shuffle = False, validation_split=0.2)

(99094, 50)
Model: "sequential_113"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_337 (LSTM)              (None, 1, 180)            166320    
_________________________________________________________________
lstm_338 (LSTM)              (None, 1, 90)             97560     
_________________________________________________________________
lstm_339 (LSTM)              (None, 30)                14520     
_________________________________________________________________
dense_113 (Dense)            (None, 1)                 31        
Total params: 278,431
Trainable params: 278,431
Non-trainable params: 0
_________________________________________________________________
Train on 79275 samples, validate on 19819 samples
Epoch 1/10
79275/79275 [==============================] - 68s 862us/step - loss: 0.7116 - accuracy: 0.5307 - val_loss: 0.6920 - val_accuracy: 0.5243
Epoch 2/10
79275/79275 [=========

KeyboardInterrupt: 

In [39]:
print('Accuracy : ', Model.evaluate(np.array(test_X),np.array(binary_test))[1])

54480/54480 [==============================] - 12s 214us/step
Accuracy :  0.5144640207290649
